In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import scipy.sparse as sp

2024-07-12 23:16:38.297130: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-12 23:16:38.597519: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 23:16:38.597589: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 23:16:38.674139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 23:16:38.772261: I tensorflow/core/platform/cpu_feature_guar

In [2]:
drugs_cell_lines_ic50_df = pd.read_csv("..//data//drugs_cell_lines_ic50.csv")

In [3]:
pubchem_drugs_smiles_df = pd.read_csv('..//data//drugs_smile_strings.csv')

In [4]:
drugs_smiles_cell_lines_ic50_df = pd.merge(drugs_cell_lines_ic50_df, pubchem_drugs_smiles_df, 
                                             on = "drug_id")

In [5]:
drugs_smiles_cell_lines_ic50_df = drugs_smiles_cell_lines_ic50_df[["drug_id", "Cancer_Cell_Line", "Smiles", "IC50"]]

In [6]:
drugs_smiles_cell_lines_ic50_df.dtypes

drug_id               int64
Cancer_Cell_Line     object
Smiles               object
IC50                float64
dtype: object

In [7]:
drugs_smiles_cell_lines_ic50_df["drug_id"] = drugs_smiles_cell_lines_ic50_df["drug_id"].astype(object)

In [8]:
drugs_smiles_cell_lines_ic50_df.shape

(186789, 4)

In [9]:
with open("..//data//drug_gcn_features.pickle", "rb") as f:
    dict_features = pickle.load(f)

In [10]:
with open("..//data//drug_gcn_normalized_adj_mats.pickle", "rb") as f:
    dict_normalized_adj_mats = pickle.load(f)

In [11]:
dualgcn_train = pd.read_csv("..//data//DualGCN_Embedding_train.csv")

In [12]:
dualgcn_test = pd.read_csv("..//data//DualGCN_Embedding_test.csv")

In [13]:
pubchem_to_drugs_df = pd.read_csv('..//data//1.Drug_listMon Jun 24 09_00_55 2019.csv')

In [14]:
pubchem_to_drugs_df = pubchem_to_drugs_df[["drug_id", "PubCHEM"]]

In [15]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [16]:
pubchem_to_drugs_df["PubCHEM"] = [val if str(val).isdigit() else np.nan for val in pubchem_to_drugs_df["PubCHEM"] ]

In [17]:
pubchem_to_drugs_df = pubchem_to_drugs_df.dropna()

In [18]:
pubchem_to_drugs_df.dtypes

drug_id     int64
PubCHEM    object
dtype: object

In [19]:
pubchem_to_drugs_df["drug_id"] = pubchem_to_drugs_df["drug_id"].astype(str)

In [20]:
x_train, x_valid, y_train, y_valid = train_test_split(drugs_smiles_cell_lines_ic50_df.drop(["IC50"],axis = 1), drugs_smiles_cell_lines_ic50_df["IC50"].values, 
                                                     test_size = 0.20, random_state = 42)

In [21]:
dualgcn_train["Drug_ID"] = dualgcn_train["Drug_ID"].astype(str)

In [22]:
dualgcn_test["Drug_ID"] = dualgcn_test["Drug_ID"].astype(str)

In [23]:
pubchem_to_drugs_df.dtypes

drug_id    object
PubCHEM    object
dtype: object

In [24]:
dualgcn_train = pubchem_to_drugs_df.merge(dualgcn_train, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [25]:
dualgcn_train = dualgcn_train[['Cell_Line', 'drug_id']]

In [26]:
dualgcn_test = pubchem_to_drugs_df.merge(dualgcn_test, left_on = ["PubCHEM"], right_on = ["Drug_ID"])

In [27]:
dualgcn_test = dualgcn_test[['Cell_Line', 'drug_id']]

In [28]:
dualgcn_train.dtypes

Cell_Line    object
drug_id      object
dtype: object

In [29]:
x_train.dtypes

drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object

In [30]:
# dualgcn_train

In [31]:
# x_train['drug_id'].values[0]

In [32]:
x_train['drug_id'] = x_train['drug_id'].astype(str)

In [33]:
x_valid['drug_id'] = x_valid['drug_id'].astype(str)

In [34]:
x_train_valid_feats = pd.concat([x_train, x_valid], ignore_index = True)

In [35]:
y_train_valid = pd.concat([pd.DataFrame(y_train.reshape(-1,1)), pd.DataFrame(y_valid.reshape(-1,1))], ignore_index = True)

In [36]:
combo_train_valid = pd.concat([x_train_valid_feats, y_train_valid],axis =  1)

In [37]:
combo_train_valid.head()

,drug_id,Cancer_Cell_Line,Smiles,0
0,1030,ACH-000295,O=c1cc(-c2cccc3c2Sc2ccccc2S3)oc(N2CCOCC2)c1,3.143044
1,203,ACH-000292,Cc1ccc2nc(NCCN)c3ncc(C)n3c2c1,2.772620
2,1072,ACH-002273,NC(=O)C(CCC(F)(F)F)N(Cc1ccc(-c2ncon2)cc1F)S(=O...,3.809977
3,150,ACH-000631,CC(O)(CS(=O)(=O)c1ccc(F)cc1)C(=O)Nc1ccc(C#N)c(...,3.053030
4,282,ACH-000270,CCOc1cc2ncc(C#N)c(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)...,-0.367467


In [38]:
combo_train_valid.columns = ['drug_id', 'Cancer_Cell_Line', 'Smiles', 'IC50']

In [39]:
# filter x_train x _valid here
x_y_train = combo_train_valid.merge(dualgcn_train, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [40]:
x_y_test = combo_train_valid.merge(dualgcn_test, left_on = ['Cancer_Cell_Line','drug_id'], right_on = [ 'Cell_Line','drug_id'])

In [41]:
x_train.dtypes

drug_id             object
Cancer_Cell_Line    object
Smiles              object
dtype: object

In [42]:
x_train, x_valid, y_train, y_valid = x_y_train.drop(["IC50", 'Cell_Line'],axis = 1), x_y_test.drop(["IC50", 'Cell_Line'],axis= 1), x_y_train["IC50"].values, x_y_test["IC50"].values

In [43]:
train_gcn_feats = []
train_adj_list = []
for drug_id in x_train["drug_id"].values:
    train_gcn_feats.append(dict_features[drug_id])
    train_adj_list.append(dict_normalized_adj_mats[drug_id])

In [44]:
valid_gcn_feats = []
valid_adj_list = []
for drug_id in x_valid["drug_id"].values:
    valid_gcn_feats.append(dict_features[drug_id])
    valid_adj_list.append(dict_normalized_adj_mats[drug_id])

In [45]:
import numpy as np

In [46]:
train_gcn_feats = np.array(train_gcn_feats).astype("float16")
valid_gcn_feats = np.array(valid_gcn_feats).astype("float16")

In [47]:
train_adj_list = np.array(train_adj_list).astype("float16")
valid_adj_list = np.array(valid_adj_list).astype("float16")

In [48]:
# load models
# omic models
cancer_copy_number_model = tf.keras.models.load_model("..//models//cancer_copy_number_model_no_norm_common")
cancer_cell_gen_expr_model = tf.keras.models.load_model("..//models//cancer_cell_gen_expr_model_no_norm_common")
cancer_cell_gen_methy_model = tf.keras.models.load_model("..//models//cancer_cell_gen_methy_model_no_norm")
cancer_cell_gen_mut_model = tf.keras.models.load_model("..//models//cancer_cell_gen_mut_model_no_norm")

2024-07-12 23:17:06.757070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0


In [49]:
# load models
# drug models
pubchem_drugs_rdkit_model = tf.keras.models.load_model("..//models//pubchem_drugs_rdkit_model_no_norm")

In [50]:
from sklearn.preprocessing import StandardScaler

In [51]:
std = StandardScaler()

In [52]:
# extract drug features
drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values).numpy().astype("float32")
drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values).numpy().astype("float32")

# drug_features_train = pubchem_drugs_rdkit_model(x_train["drug_id"].values)
# drug_features_valid = pubchem_drugs_rdkit_model(x_valid["drug_id"].values)

In [53]:
np.isinf(drug_features_train).sum()

0

In [54]:
drug_features_train = std.fit_transform(drug_features_train)

In [55]:
drug_features_valid = std.transform(drug_features_valid)

In [56]:
# extract copy number features
omics_copy_number_train = cancer_copy_number_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_copy_number_valid = cancer_copy_number_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [57]:
# extract gen expr features
omics_gen_expr_train = cancer_cell_gen_expr_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_expr_valid = cancer_cell_gen_expr_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [58]:
# extract gen methylation features
omics_gen_methyl_train = cancer_cell_gen_methy_model(x_train["Cancer_Cell_Line"].values).numpy().astype("float16")
omics_gen_methyl_valid = cancer_cell_gen_methy_model(x_valid["Cancer_Cell_Line"].values).numpy().astype("float16")

In [59]:
# extract gen mutation features
with tf.device('/cpu:0'):
    omics_gen_mut_train = cancer_cell_gen_mut_model.predict(x_train["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")
    omics_gen_mut_valid = cancer_cell_gen_mut_model.predict(x_valid["Cancer_Cell_Line"].values, verbose = 1, batch_size = 256).astype("float16")

68/68 [==============================] - 0s 5ms/step


In [60]:
smile_strings_train = x_train["Smiles"].values.reshape(-1,1)
smile_strings_valid = x_valid["Smiles"].values.reshape(-1,1)

In [61]:
input_gcn_features = tf.keras.layers.Input(shape = (100, 75))
input_norm_adj_mat = tf.keras.layers.Input(shape = (100, 100))
mult_1 = tf.keras.layers.Dot(1)([input_norm_adj_mat, input_gcn_features])
dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
dense_out = dense_layer_gcn(mult_1)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)
mult_2 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
dense_layer_gcn = tf.keras.layers.Dense(256, activation = "relu")
dense_out = dense_layer_gcn(mult_2)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)

dense_layer_gcn = tf.keras.layers.Dense(100, activation = "relu")
mult_3 = tf.keras.layers.Dot(1)([input_norm_adj_mat, dense_out])
dense_out = dense_layer_gcn(mult_3)
dense_out = tf.keras.layers.BatchNormalization()(dense_out)
dense_out = tf.keras.layers.Dropout(0.2)(dense_out)

dense_out = tf.keras.layers.GlobalAvgPool1D()(dense_out)

In [62]:
input_gen_methy = tf.keras.layers.Input(shape = (omics_gen_methyl_train.shape[1],))

In [63]:
gen_methy_layer = tf.keras.layers.Dense(256, activation = "tanh")

In [64]:
gen_methy_emb = gen_methy_layer(input_gen_methy)
gen_methy_emb = tf.keras.layers.BatchNormalization()(gen_methy_emb)
gen_methy_emb = tf.keras.layers.Dropout(0.2)(gen_methy_emb)

In [65]:
gen_methy_layer = tf.keras.layers.Dense(100, activation = "relu")
gen_methy_emb = gen_methy_layer(gen_methy_emb)

In [66]:
all_feats = tf.keras.layers.Concatenate()([gen_methy_emb, dense_out])

In [67]:
x = tf.keras.layers.Dense(300,activation = 'tanh')(all_feats)
x = tf.keras.layers.Dropout(0.2)(x, training = True)
x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=-1))(x)
x = tf.keras.layers.Lambda(lambda x: K.expand_dims(x,axis=1))(x)
x = tf.keras.layers.Conv2D(filters=30, kernel_size=(1,150),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,2))(x)
x = tf.keras.layers.Conv2D(filters=10, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
x = tf.keras.layers.Conv2D(filters=5, kernel_size=(1,5),strides=(1, 1), activation = 'relu',padding='valid')(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(1,3))(x)
x = tf.keras.layers.Dropout(0.2)(x, training = True)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x, training = True)

In [68]:
final_out_layer = tf.keras.layers.Dense(1)

In [69]:
final_out = final_out_layer(x)

In [70]:
simplecdr = tf.keras.models.Model([input_gcn_features, input_norm_adj_mat,
                                   input_gen_methy], final_out)

In [71]:
simplecdr.compile(loss = tf.keras.losses.MeanSquaredError(), 
                    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
                    metrics = [tf.keras.metrics.RootMeanSquaredError()])

In [72]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [73]:
chunks = np.array_split(list(range(0,valid_gcn_feats.shape[0])), 3)

In [74]:
test_index = 0

In [75]:
train_items = list(chunks[test_index])

In [76]:
test_items = list(set(list(range(0,valid_gcn_feats.shape[0]))).difference(train_items))

In [77]:
# First the data - let's use a train set of 6k - then the rest will be test here (from the remaining values, use 3k for training the stacker, 1k for validation of it, and the rest ~7k as the final test data). 
# drug feats
new_train_gcn_feats = valid_gcn_feats[train_items, :,:]
new_test_gcn_feats = valid_gcn_feats[test_items, :,:]
print(new_train_gcn_feats.shape, new_test_gcn_feats.shape)

# drug adj info
new_train_adj_list = valid_adj_list[train_items, :,:]
new_test_adj_list = valid_adj_list[test_items, :,:]
print(new_train_adj_list.shape, new_test_adj_list.shape)

# expression
new_omics_gen_expr_train = omics_gen_expr_valid[train_items, :]
new_omics_gen_expr_test = omics_gen_expr_valid[test_items,:]
print(new_omics_gen_expr_train.shape, new_omics_gen_expr_test.shape)

# methylation
new_omics_gen_methyl_train = omics_gen_methyl_valid[train_items, :]
new_omics_gen_methyl_test = omics_gen_methyl_valid[test_items,:]
print(new_omics_gen_methyl_train.shape, new_omics_gen_methyl_test.shape)

# mutation
new_omics_gen_mut_train = omics_gen_mut_valid[train_items, :]
new_omics_gen_mut_test = omics_gen_mut_valid[test_items,:]
print(new_omics_gen_mut_train.shape, new_omics_gen_mut_test.shape)

# y
new_y_train = y_valid[train_items,]
new_y_test = y_valid[test_items,]
print(new_y_train.shape, new_y_test.shape)

(5772, 100, 75) (11544, 100, 75)
(5772, 100, 100) (11544, 100, 100)
(5772, 691) (11544, 691)
(5772, 808) (11544, 808)
(5772, 34673) (11544, 34673)
(5772,) (11544,)


In [78]:
new_y_train.mean(), new_y_train.std() # match

(2.069332453915454, 2.8460217894549977)

In [79]:
simpleCDR_original_model = tf.keras.models.load_model("../models/deepcdr_trained_on_domain")
simpleCDR_original_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 100, 75)]            0         []                            
                                                                                                  
 dot (Dot)                   (None, 100, 75)              0         ['input_2[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 100, 256)             19456     ['dot[0][0]']             

In [80]:
simplecdr.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 100, 75)]            0         []                            
                                                                                                  
 dot (Dot)                   (None, 100, 75)              0         ['input_2[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 100, 256)             19456     ['dot[0][0]']             

In [81]:
# unpack the weights for the warm start

# first drugs
Dense_1_weights, Dense_1_bias = simpleCDR_original_model.layers[3].get_weights()
bn_1_weights = simpleCDR_original_model.layers[4].get_weights()
Dense_2_weights, Dense_2_bias = simpleCDR_original_model.layers[7].get_weights()
bn_2_weights = simpleCDR_original_model.layers[9].get_weights()
Dense_3_weights, Dense_3_bias = simpleCDR_original_model.layers[19].get_weights()
bn_3_weights = simpleCDR_original_model.layers[23].get_weights()

# # methylation
Dense_1_weights_methyl, Dense_1_bias_methyl = simpleCDR_original_model.layers[16].get_weights()
bn_1_weights_methyl = simpleCDR_original_model.layers[20].get_weights()
Dense_2_weights_methyl, Dense_2_bias_methyl = simpleCDR_original_model.layers[28].get_weights()

# once concatenated
conv2d_1_weights_concat, conv2d_1_bias_concat = simpleCDR_original_model.layers[37].get_weights()
conv2d_2_weights_concat, conv2d_2_bias_concat = simpleCDR_original_model.layers[39].get_weights()
conv2d_3_weights_concat, conv2d_3_bias_concat = simpleCDR_original_model.layers[41].get_weights()
Dense_final_weights_concat, Dense_final_bias_concat = simpleCDR_original_model.layers[46].get_weights()

In [82]:
simpleCDR_original_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 input_1 (InputLayer)        [(None, 100, 75)]            0         []                            
                                                                                                  
 dot (Dot)                   (None, 100, 75)              0         ['input_2[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 dense (Dense)               (None, 100, 256)             19456     ['dot[0][0]']             

In [83]:
# Set weights

# drugs
simplecdr.layers[3].set_weights((Dense_1_weights, Dense_1_bias))
simplecdr.layers[4].set_weights(bn_1_weights)
simplecdr.layers[7].set_weights((Dense_2_weights, Dense_2_bias))
simplecdr.layers[8].set_weights(bn_2_weights)
simplecdr.layers[13].set_weights((Dense_3_weights, Dense_3_bias))
simplecdr.layers[15].set_weights(bn_3_weights)

# methylation
simplecdr.layers[12].set_weights((Dense_1_weights_methyl, Dense_1_bias_methyl))
simplecdr.layers[14].set_weights(bn_1_weights_methyl)
simplecdr.layers[18].set_weights((Dense_2_weights_methyl, Dense_2_bias_methyl))

# concat
simplecdr.layers[25].set_weights((conv2d_1_weights_concat, conv2d_1_bias_concat))
simplecdr.layers[27].set_weights((conv2d_2_weights_concat, conv2d_2_bias_concat))
simplecdr.layers[29].set_weights((conv2d_3_weights_concat, conv2d_3_bias_concat))
simplecdr.layers[34].set_weights((Dense_final_weights_concat, Dense_final_bias_concat))

In [84]:
%%time
history = simplecdr.fit([new_train_gcn_feats, new_train_adj_list,
                         new_omics_gen_methyl_train], new_y_train.reshape(-1,1), 
                         
          batch_size = 512, epochs = 1000, verbose = 1,
                         
          validation_split=0.2,
        callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 20, restore_best_weights=True,
                                                       mode = "min"), 
         validation_batch_size = 512, shuffle = True)

Epoch 1/1000


2024-07-12 23:17:40.374381: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout_5/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-07-12 23:17:42.130538: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-07-12 23:17:42.811718: I external/local_xla/xla/service/service.cc:168] XLA service 0x146cf54e5f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-12 23:17:42.811774: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2024-07-12 23:17:42.862036: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1720844263.153757  343720 device_compiler.h:186] Compiled cluster using XLA!  T

10/10 [==============================] - 9s 101ms/step - loss: 5.5154 - root_mean_squared_error: 2.3485 - val_loss: 3.3249 - val_root_mean_squared_error: 1.8234
Epoch 2/1000
10/10 [==============================] - 0s 37ms/step - loss: 3.3073 - root_mean_squared_error: 1.8186 - val_loss: 3.0535 - val_root_mean_squared_error: 1.7474
Epoch 3/1000
10/10 [==============================] - 0s 38ms/step - loss: 2.7209 - root_mean_squared_error: 1.6495 - val_loss: 2.6087 - val_root_mean_squared_error: 1.6152
Epoch 4/1000
10/10 [==============================] - 0s 38ms/step - loss: 2.5530 - root_mean_squared_error: 1.5978 - val_loss: 2.5737 - val_root_mean_squared_error: 1.6043
Epoch 5/1000
10/10 [==============================] - 0s 37ms/step - loss: 2.4428 - root_mean_squared_error: 1.5629 - val_loss: 2.7426 - val_root_mean_squared_error: 1.6561
Epoch 6/1000
10/10 [==============================] - 0s 38ms/step - loss: 2.2938 - root_mean_squared_error: 1.5145 - val_loss: 2.4447 - val_root_m

In [85]:
simplecdr.save("..//models//ablation_deepcdr_fine_tuned_split_" + str(test_index))

INFO:tensorflow:Assets written to: ..//models//ablation_deepcdr_fine_tuned_split_0/assets


INFO:tensorflow:Assets written to: ..//models//ablation_deepcdr_fine_tuned_split_0/assets


In [86]:
reps = 25

In [87]:
catch_preds = np.zeros((new_test_gcn_feats.shape[0],reps))
for i in range(0,reps):
    print(i)
    preds = simplecdr.predict([ new_test_gcn_feats, new_test_adj_list,
                           new_omics_gen_methyl_test], 
                              batch_size = 1024, verbose = 0)
    catch_preds[:,i] = preds[:,0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [88]:
preds_data = np.hstack((new_y_test.reshape(-1,1), catch_preds))

In [89]:
np.save("..//data//ablation_deepcdr_preds_val_and_test_" +str(test_index) + ".npy",preds_data )